In [1]:
from glob import glob
import labelme2coco
import os
import shutil
import json
import numpy as np
from tqdm import tqdm

### Clean data annotations

Move all the annotations file to one folder

In [2]:
data_folders = glob(os.getcwd() + '/../data/birds/annotations/all/*')
data_files = glob(os.getcwd() + '/../data/birds/annotations/22*/*')
len(data_folders), len(data_files)

(2194, 2194)

In [ ]:
def CopyAnno(data_files, data_folders):

    out_fns = [f.split('/')[-1] for f in data_folders]
    out_dir = os.getcwd() + '/../data/birds/annotations/all'
    n = 0

    for p in tqdm(data_files):
        in_fn = p.split('/')[-1]
        if in_fn not in out_fns:
            n += 1
            shutil.copy(p, out_dir)
    if n!=0:
        print('Done copying', n, ' files.')
    else:
        print('No files copied.')

Convert filepaths to apply labelme2coco functions

In [ ]:
data_folders = glob(os.getcwd() + '/../data/birds/annotations/all/*')

for d in tqdm(data_folders):
    with open(d, "r") as f:
        anno_json = json.load(f)
        anno_json['imagePath'] = anno_json['imagePath'].replace('\\','/')

    with open(d, "w") as f:
        json_string = json.dumps(anno_json)
        f.write(json_string)

In [ ]:
# import functions
from labelme2coco import get_coco_from_labelme_folder, save_json

# set labelme training data directory
labelme_folder = os.getcwd() + '/../data/birds/annotations/all'

# convert labelme annotations to coco
out = get_coco_from_labelme_folder(labelme_folder)

In [ ]:
# set path for coco json to be saved
export_dir = os.getcwd() + '/../data/birds/'

# export train coco json
save_json(out.json, export_dir + "coco_annotations.json")

## Data cleaning

In [2]:
path_in = os.getcwd() + '/../data/birds/annotations/coco_annotations.json'

with open(path_in, "r") as f:
    anno = json.load(f)


In [4]:
anno.keys()

dict_keys(['images', 'annotations', 'categories'])

In [7]:
anno['images'][0].keys()

dict_keys(['height', 'width', 'id', 'file_name'])

In [8]:
anno['annotations'][0].keys()

dict_keys(['iscrowd', 'image_id', 'bbox', 'segmentation', 'category_id', 'id', 'area'])

In [79]:
anno['categories']

[{'id': 0, 'name': 'bird', 'supercategory': 'bird'},
 {'id': 1, 'name': 'car', 'supercategory': 'car'},
 {'id': 2, 'name': 'dog', 'supercategory': 'dog'},
 {'id': 3, 'name': 'human', 'supercategory': 'human'}]

Remove the images with annotations other than bird: 'car', 'dog', 'human'.

    (1) find images containing the annotations
    (2) find if birds are on these images
    (3) remove the annotations

In [30]:
anno_id = [a['category_id'] for a in anno['annotations']]
img_id = [a['image_id'] for a in anno['annotations']]

In [53]:
cond = [a != 0 for a in anno_id]
print('There are ', np.count_nonzero(cond), 'annotations not bird.')
cat, count = np.unique(np.array(anno_id)[cond], return_counts=True)
print('Other annotations of', count[0],'"car",', count[1],'"dog",', count [2],'"human".')

img_id_other = np.array(img_id)[cond]
print('Image IDs ', img_id_other)

There are  6 annotations not bird.
Other annotations of 3 "car", 1 "dog", 2 "human".
Image IDs  [ 295  296  778 1215 1216 1525]


In [71]:
## Remove Annotations other than birds
anno_list = [a for a in anno['annotations'] if a['category_id'] == 0]

## Remove images containing no annotations
img_ids = np.unique([a['image_id'] for a in anno_list])
img_list = [a for a in anno['images'] if a['id'] in img_ids]

In [73]:
len(anno['images']), len(img_list)

(2194, 2186)

In [78]:
len(np.unique(img_ids)), len(np.unique(img_id))

(2186, 2191)

In [82]:
anno['categories'][0]

{'id': 0, 'name': 'bird', 'supercategory': 'bird'}

In [88]:
new_anno = {}
new_anno['images'] = img_list
new_anno['annotations'] = anno_list
new_anno['categories'] = [{'id':0, 'name':'bird'}]

In [89]:
with open(path_in, "w") as f:
    json_string = json.dumps(new_anno)
    f.write(json_string)

# Create empty annotations

sourche info create null anno: https://blog.roboflow.com/missing-and-null-image-annotations/

source info distribution: https://towardsdatascience.com/the-practical-guide-for-object-detection-with-yolov5-algorithm-74c04aac4843

In [3]:
path_full= os.getcwd() + '/../data/birds/annotations/coco_annotations.json'
path_test = os.getcwd() + '/../data/birds/annotations/coco_test_dataset.json'
path_train = os.getcwd() + '/../data/birds/annotations/coco_train_dataset.json'
path_val = os.getcwd() + '/../data/birds/annotations/coco_val_dataset.json'

date_list = ['220430', '220508', '220502','220510']

with open(path_full, "r") as f:
    data = json.load(f)
with open(path_test, "r") as f:
    data_test = json.load(f)
with open(path_train, "r") as f:
    data_train = json.load(f)
with open(path_val, "r") as f:
    data_val = json.load(f)

In [4]:
fns = [f['file_name'].split('/')[-1] for f in data['images']]
last_id = data['images'][-1]['id']
h_dict = {d['file_name'].split('/')[-1][:8]:d['height'] for d in data['images']}
w_dict = {d['file_name'].split('/')[-1][:8]:d['width'] for d in data['images']}

fold_dict = {'C5_AA_SD__220501':'220501_bis/',
            'C5_AP_SN__220501':'220501_bis/',
            'C3_AP_BF__220503':'220503_bis/',
            'C3_AP_SD__220503':'220503_bis/',
            'C8_AA_BF__220507':'220507_bis/',
            'C8_AP_ST__220507':'220507_bis/',
            'C7_AA_BF__220509':'220509_bis/',
            'C7_AP_ST__220509':'220509_bis/',
            'C6_AA_SD__220511':'220511_bis/',
            'C6_AP_SN__220511':'220511_bis/'
            }

In [6]:
fns_all_paths = glob(os.getcwd() + '/../data/birds/images/22*/*.png')
fns_all = [f.split('/')[-1] for f in fns_all_paths]
fns_new = [f for f in fns_all if f not in fns]

## select test dataset
dates = [e.split('_')[-2] for e in fns_new]
cond = [d in date_list for d in dates]

## create new 'file_name'
filenames = ['../../images/' + f.split('_')[-2]+ '/' + f for f in fns_new]

for i, f in enumerate(filenames):
    
    if f.split('/')[-1][:-10] in fold_dict.keys():
        new_fn_val = f.replace(f.split('_')[-2]+'/', fold_dict[f.split('/')[-1][:-10]])
        filenames[i] = new_fn_val

filenames_test = np.array(filenames)[cond]
np.random.shuffle(filenames_test)

filenames_train = np.array(filenames)[np.invert(cond)]
np.random.shuffle(filenames_train)

filenames_val = filenames_train[:-10000]

filenames_train = filenames_train[:10000]

In [8]:
len(filenames_test)

7196

In [11]:
def empty_test(fn_list, h_dict, w_dict):
    
    id_num = 0

    print('Creating ', len(fn_list), ' new empty annotations')
    new_dict = {'images':[],
                'annotations':[],
                'categories': []}
    

    for n in range(len(fn_list)):

        img_dict = {}
        id_num += 1
        ref = fn_list[n].split('/')[-1][:8]

        img_dict['height'] = h_dict[ref]
        img_dict['width'] = w_dict[ref]
        img_dict['id'] = id_num
        img_dict['file_name'] = fn_list[n]
        
        new_dict['images'].append(img_dict)
        
    return new_dict

Create 10% empty annotations.

In [5]:
def empty_anno(data, fn_list, last_id, h_dict, w_dict, p):

    num_anno = int(len(data['images'])*p/(1-p))
    id_num = last_id

    print('Creating ', num_anno, ' new empty annotations')
    

    for n in range(num_anno):

        img_dict = {}
        id_num += 1
        ref = fn_list[n].split('/')[-1][:8]

        img_dict['height'] = h_dict[ref]
        img_dict['width'] = w_dict[ref]
        img_dict['id'] = id_num
        img_dict['file_name'] = fn_list[n]
        
        data['images'].append(img_dict)

In [8]:
empty_anno(data_test, filenames_test, last_id, h_dict, w_dict, 0.2)

last_id = data_test['images'][-1]['id']
empty_anno(data_train, filenames_train, last_id, h_dict, w_dict, 0.2)

last_id = data_train['images'][-1]['id']
empty_anno(data_val, filenames_val, last_id, h_dict, w_dict, 0.2)

Creating  61  new empty annotations
Creating  431  new empty annotations
Creating  54  new empty annotations


In [12]:
out_test = empty_test(filenames_test, h_dict, w_dict)

Creating  7196  new empty annotations


In [17]:
out_test['categories'] = data['categories']

In [18]:
## save new datasets
with open(os.getcwd() + '/../data/birds/annotations/empty_test_dataset.json', "w") as f:
    json_string = json.dumps(out_test)
    f.write(json_string)

In [9]:
## save new datasets
with open(os.getcwd() + '/../data/birds/annotations/full_coco_test.json', "w") as f:
    json_string = json.dumps(data_test)
    f.write(json_string)
with open(os.getcwd() + '/../data/birds/annotations/full_coco_train.json', "w") as f:
    json_string = json.dumps(data_train)
    f.write(json_string)
with open(os.getcwd() + '/../data/birds/annotations/full_coco_val.json', "w") as f:
    json_string = json.dumps(data_val)
    f.write(json_string)